# Q1 - R section

In [1]:
library(limma)
library(edgeR)

In [2]:
normal_counts <- read.csv("train_normal_counts.csv")

In [3]:
head(normal_counts)

,DLDR_0036,DLDR_0081,DLDR_0191,DLDR_0188,DLDR_0130,DLDR_0013,DLDR_0079,DLDR_0131,DLDR_0135,DLDR_0190,⋯,DLDR_0175,DLDR_0052,DLDR_0087,DLDR_0155,DLDR_0092,DLDR_0187,DLDR_0186,DLDR_0179,DLDR_0182,DLDR_0001
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,5.820135,6.5462994,6.6040504,6.480745,6.550016,6.5692529,5.9652146,6.436726,6.3378589,6.3677048,⋯,6.85952138,6.3703867,6.6352203,6.693334,6.26633823,6.6089237,6.34423435,6.296354,6.485184,5.965571
2,-1.060061,0.5821648,-0.8650363,-1.083676,-1.222374,0.7672549,-0.2056643,1.225274,0.3287041,-0.9022011,⋯,0.03594184,-0.5651604,-0.4658174,1.603149,0.08898988,-0.7573986,-0.02658513,-2.568864,-0.486609,1.612375
3,4.388400,3.7520898,4.3514891,4.361634,4.534941,4.1504701,3.1093774,4.104779,4.6099902,4.1480104,⋯,4.12340468,4.7392966,4.3778794,4.473400,4.31945643,4.2744503,4.17931917,4.381449,4.331904,4.133821
4,4.080172,4.6451746,4.0721368,4.313540,4.370763,4.1660389,4.8092861,4.411547,4.3253607,4.5060579,⋯,4.76204644,4.1208213,4.2298584,4.755317,4.38806704,4.0425768,4.43879576,4.365810,4.090491,4.111056
5,2.564430,3.8408991,3.1431376,3.120196,3.512952,3.7570108,3.9527651,3.099743,3.3085263,3.9820712,⋯,3.95559973,2.9326175,3.6972213,4.027257,3.97372734,2.1264084,3.81471713,2.965115,2.697188,4.150662
6,3.552685,3.2010747,4.0374758,1.941859,2.517867,3.2536535,2.7857226,3.357725,1.9136666,2.2021356,⋯,2.72896408,2.8246104,2.2057995,1.576431,2.97651515,3.4239312,2.27774891,3.408416,3.630074,2.975845


In [4]:
dim(normal_counts)

[1] 17396   134

In [5]:
meta_data <- read.csv("train_meta_data.csv")

In [6]:
head(meta_data)

,Simplified_class
,<chr>
1,Normal
2,Advanced_fibrosis
3,Normal
4,Normal
5,Non_advanced_Fibrosis
6,Normal


In [7]:
dim(meta_data)

[1] 134   1

In [8]:
labels <- factor(meta_data$Simplified_class)

In [9]:
print(labels)

  [1] Normal                Advanced_fibrosis     Normal               
  [4] Normal                Non_advanced_Fibrosis Normal               
  [7] Advanced_fibrosis     Non_advanced_Fibrosis Non_advanced_Fibrosis
 [10] Normal                Advanced_fibrosis     Advanced_fibrosis    
 [13] Advanced_fibrosis     Non_advanced_Fibrosis Advanced_fibrosis    
 [16] Advanced_fibrosis     Normal                Normal               
 [19] Normal                Normal                Advanced_fibrosis    
 [22] Advanced_fibrosis     Non_advanced_Fibrosis Non_advanced_Fibrosis
 [25] Advanced_fibrosis     Non_advanced_Fibrosis Normal               
 [28] Non_advanced_Fibrosis Normal                Advanced_fibrosis    
 [31] Advanced_fibrosis     Advanced_fibrosis     Advanced_fibrosis    
 [34] Normal                Normal                Non_advanced_Fibrosis
 [37] Non_advanced_Fibrosis Advanced_fibrosis     Advanced_fibrosis    
 [40] Non_advanced_Fibrosis Advanced_fibrosis     Advanced_fibro

In [10]:
# Create a design matrix
design <- model.matrix(~0 + labels)
colnames(design) <- levels(labels)

In [11]:
fit <- lmFit(normal_counts, design)

In [12]:
contrast.matrix <- makeContrasts(
    AdvancedFibrosis_vs_Normal = `Advanced_fibrosis` - Normal,
    Fibrosis_vs_Normal = Non_advanced_Fibrosis - Normal,
    AdvancedFibrosis_vs_Fibrosis = `Advanced_fibrosis` - Non_advanced_Fibrosis,
    levels = design
)

# Apply contrasts to the fit
fit2 <- contrasts.fit(fit, contrast.matrix)

# Empirical Bayes moderation to get p-values
fit2 <- eBayes(fit2)

In [13]:
# Get the top DEGs for the Advanced Fibrosis vs Normal comparison
top_genes_adv_vs_norm <- topTable(fit2, coef = "AdvancedFibrosis_vs_Normal", adjust.method = "BH", number = Inf)

# Get the top DEGs for the Fibrosis vs Normal comparison
top_genes_fib_vs_norm <- topTable(fit2, coef = "Fibrosis_vs_Normal", adjust.method = "BH", number = Inf)

# Get the top DEGs for the Advanced Fibrosis vs Fibrosis comparison
top_genes_adv_vs_fib <- topTable(fit2, coef = "AdvancedFibrosis_vs_Fibrosis", adjust.method = "BH", number = Inf)

# View the top DEGs
head(top_genes_adv_vs_norm)
head(top_genes_fib_vs_norm)
head(top_genes_adv_vs_fib)

,logFC,AveExpr,t,P.Value,adj.P.Val,B
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
10728,-1.3278914,1.902497,-10.235955,1.733536e-18,3.015659e-14,31.41974
13385,1.0088994,6.051398,9.886439,1.299982e-17,9.020981e-14,29.46392
10694,-1.3132140,3.432308,-9.855214,1.555699e-17,9.020981e-14,29.28958
16113,-3.4233202,-1.392599,-9.734863,3.105946e-17,1.350776e-13,28.61834
16278,-2.8530739,-0.172656,-9.680406,4.245065e-17,1.476943e-13,28.31501
6969,0.4379282,4.207436,9.600224,6.721385e-17,1.948753e-13,27.86885


,logFC,AveExpr,t,P.Value,adj.P.Val,B
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
13623,0.7946468,5.255822,11.73980,2.831866e-22,4.926315e-18,40.01880
10970,0.6321228,5.361546,11.47202,1.339959e-21,1.021819e-17,38.49831
5442,-1.4931333,2.180779,-11.38733,2.190883e-21,1.021819e-17,38.01728
17075,0.6202635,7.265926,11.37529,2.349550e-21,1.021819e-17,37.94887
4461,0.6782287,6.569435,11.29235,3.802820e-21,1.323077e-17,37.47774
6563,0.6004911,5.545244,11.13147,9.676618e-21,2.805574e-17,36.56387


,logFC,AveExpr,t,P.Value,adj.P.Val,B
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
16863,1.223013,2.9208664,9.097265,1.182040e-15,2.056276e-11,24.90970
3296,1.594761,1.9955120,8.122841,2.755851e-13,1.021684e-09,19.70546
673,1.485822,1.1568774,8.121636,2.774199e-13,1.021684e-09,19.69912
14913,-1.153029,2.6115748,-8.115173,2.874679e-13,1.021684e-09,19.66515
12060,1.675825,0.1951384,8.111304,2.936548e-13,1.021684e-09,19.64482
3227,1.279852,4.3441375,8.016339,4.947792e-13,1.434530e-09,19.14676


In [14]:
write.csv(top_genes_adv_vs_norm, "DEGs_AdvancedFibrosis_vs_Normal.csv")
write.csv(top_genes_fib_vs_norm, "DEGs_Fibrosis_vs_Normal.csv")
write.csv(top_genes_adv_vs_fib, "DEGs_AdvancedFibrosis_vs_Fibrosis.csv")

In [15]:
filtered_genes_adv_vs_norm <- top_genes_adv_vs_norm[1:200,]
filtered_genes_fib_vs_norm <- top_genes_fib_vs_norm[1:200,]
filtered_genes_adv_vs_fib <- top_genes_adv_vs_fib[1:200,]

# View filtered DEGs
head(filtered_genes_adv_vs_norm)
head(filtered_genes_fib_vs_norm)
head(filtered_genes_adv_vs_fib)

,logFC,AveExpr,t,P.Value,adj.P.Val,B
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
10728,-1.3278914,1.902497,-10.235955,1.733536e-18,3.015659e-14,31.41974
13385,1.0088994,6.051398,9.886439,1.299982e-17,9.020981e-14,29.46392
10694,-1.3132140,3.432308,-9.855214,1.555699e-17,9.020981e-14,29.28958
16113,-3.4233202,-1.392599,-9.734863,3.105946e-17,1.350776e-13,28.61834
16278,-2.8530739,-0.172656,-9.680406,4.245065e-17,1.476943e-13,28.31501
6969,0.4379282,4.207436,9.600224,6.721385e-17,1.948753e-13,27.86885


,logFC,AveExpr,t,P.Value,adj.P.Val,B
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
13623,0.7946468,5.255822,11.73980,2.831866e-22,4.926315e-18,40.01880
10970,0.6321228,5.361546,11.47202,1.339959e-21,1.021819e-17,38.49831
5442,-1.4931333,2.180779,-11.38733,2.190883e-21,1.021819e-17,38.01728
17075,0.6202635,7.265926,11.37529,2.349550e-21,1.021819e-17,37.94887
4461,0.6782287,6.569435,11.29235,3.802820e-21,1.323077e-17,37.47774
6563,0.6004911,5.545244,11.13147,9.676618e-21,2.805574e-17,36.56387


,logFC,AveExpr,t,P.Value,adj.P.Val,B
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
16863,1.223013,2.9208664,9.097265,1.182040e-15,2.056276e-11,24.90970
3296,1.594761,1.9955120,8.122841,2.755851e-13,1.021684e-09,19.70546
673,1.485822,1.1568774,8.121636,2.774199e-13,1.021684e-09,19.69912
14913,-1.153029,2.6115748,-8.115173,2.874679e-13,1.021684e-09,19.66515
12060,1.675825,0.1951384,8.111304,2.936548e-13,1.021684e-09,19.64482
3227,1.279852,4.3441375,8.016339,4.947792e-13,1.434530e-09,19.14676


In [16]:
dim(filtered_genes_adv_vs_norm)
dim(filtered_genes_fib_vs_norm)
dim(filtered_genes_adv_vs_fib)

[1] 200   6

[1] 200   6

[1] 200   6

In [17]:
genes_adv_vs_norm_names <- rownames(filtered_genes_adv_vs_norm)
genes_fib_vs_norm_names <- rownames(filtered_genes_fib_vs_norm)
genes_adv_vs_fib_names <- rownames(filtered_genes_adv_vs_fib)

In [18]:
combined_gene_names <- unique(c(genes_adv_vs_norm_names,
                                genes_fib_vs_norm_names,
                                genes_adv_vs_fib_names))

In [19]:
length(combined_gene_names)

[1] 527

In [20]:
common_genes <- intersect(rownames(normal_counts), combined_gene_names)
selected_normal_counts <- normal_counts[common_genes, ]
head(selected_normal_counts)

,DLDR_0036,DLDR_0081,DLDR_0191,DLDR_0188,DLDR_0130,DLDR_0013,DLDR_0079,DLDR_0131,DLDR_0135,DLDR_0190,⋯,DLDR_0175,DLDR_0052,DLDR_0087,DLDR_0155,DLDR_0092,DLDR_0187,DLDR_0186,DLDR_0179,DLDR_0182,DLDR_0001
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
10,4.5895546,5.4821690,5.01315395,5.0660709,4.6558168,4.29907767,4.752957,5.4095139,4.9937768,4.5285302,⋯,4.853565,4.5671727,4.9851582,5.2347762,4.9007545,4.9230866,5.0044999,5.0580621,5.0024535,4.221449898
57,-0.7190239,0.5821648,-1.15454295,-1.0836762,-0.2894877,-0.00646928,-1.138550,-4.0601279,-2.1042553,-1.3496601,⋯,-2.285986,-0.1105945,-0.3856470,-3.5800732,0.3172589,-0.7573986,-3.4860167,-1.3464713,-1.1032804,-0.005377495
265,0.5876374,-2.2251901,-0.07654044,0.1964317,0.2723912,2.52315019,1.424386,0.3321895,0.2176728,0.1451046,⋯,1.083248,0.3523775,-0.3096982,0.1203665,0.8904442,-0.2917350,0.8116638,-0.2469356,0.4160938,2.729098547
275,1.6624052,2.5296974,1.50842206,1.9916119,1.4679420,1.93415217,2.031375,2.0483966,0.5280129,1.6338518,⋯,1.083248,2.5564691,2.0559513,1.3741231,1.9022214,1.8389687,1.6432663,1.2724386,1.9323435,2.467558388
278,5.5299023,5.8462723,5.55358296,5.3592673,5.7745493,4.96266190,5.201300,5.5002050,5.7820510,5.4935472,⋯,5.862067,5.0386033,5.6652311,5.7811395,5.7451593,5.4380615,5.5701694,5.5587008,5.7338219,5.220120837
297,7.5494873,7.7592284,7.61253797,8.0700567,7.4394046,7.67164807,7.481170,7.5568804,8.3222192,7.5618681,⋯,7.462766,8.3865560,8.5813066,7.9407091,8.7308868,8.0046275,7.7268849,7.2454517,7.7264423,8.111235350


In [22]:
dim(selected_normal_counts)

[1] 527 134

In [23]:
write.csv(selected_normal_counts, "subset_data.csv")